In [1]:
from untils import load_data
import numpy as np

In [2]:
data_file = 'D:/zhuomian/file/弱网中鲁棒性的恶意节点检测/exp/dataset/20241106_testTX_constantUniform_simtime-100_num_nodes-50_BHradio-0_SFradio-0_fieldLength-20_uniform-1/ProcessedMonitorSnifferRx.csv'

In [3]:
processed_data = load_data(1,data_file)

{(1.0, 1.0): 0, (19.0, 1.0): 1, (20.0, 1.0): 2, (24.0, 1.0): 3, (25.0, 1.0): 4, (26.0, 1.0): 5, (27.0, 1.0): 6, (28.0, 1.0): 7, (32.0, 1.0): 8, (33.0, 1.0): 9, (34.0, 1.0): 10, (2.0, 2.0): 11, (3.0, 2.0): 12, (4.0, 2.0): 13, (9.0, 2.0): 14, (10.0, 2.0): 15, (16.0, 2.0): 16, (2.0, 3.0): 17, (3.0, 3.0): 18, (4.0, 3.0): 19, (5.0, 3.0): 20, (9.0, 3.0): 21, (10.0, 3.0): 22, (11.0, 3.0): 23, (17.0, 3.0): 24, (2.0, 4.0): 25, (3.0, 4.0): 26, (4.0, 4.0): 27, (5.0, 4.0): 28, (10.0, 4.0): 29, (11.0, 4.0): 30, (12.0, 4.0): 31, (18.0, 4.0): 32, (3.0, 5.0): 33, (4.0, 5.0): 34, (5.0, 5.0): 35, (6.0, 5.0): 36, (7.0, 5.0): 37, (11.0, 5.0): 38, (12.0, 5.0): 39, (13.0, 5.0): 40, (19.0, 5.0): 41, (4.0, 6.0): 42, (5.0, 6.0): 43, (6.0, 6.0): 44, (7.0, 6.0): 45, (8.0, 6.0): 46, (12.0, 6.0): 47, (13.0, 6.0): 48, (14.0, 6.0): 49, (20.0, 6.0): 50, (5.0, 7.0): 51, (6.0, 7.0): 52, (7.0, 7.0): 53, (8.0, 7.0): 54, (13.0, 7.0): 55, (14.0, 7.0): 56, (15.0, 7.0): 57, (21.0, 7.0): 58, (6.0, 8.0): 59, (7.0, 8.0): 60, (8

In [4]:
# 设置numpy的打印选项，显示更多行列
np.set_printoptions(threshold=np.inf)  # 这将打印所有内容，去掉折叠

# 打印节点特征（processed_data.x）
print(processed_data.x.numpy())

[[  0.   0.   0.   0.   0.   0.   0.   8.]
 [  0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   4.   3.   0.  10.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   3.   3.   0.   9.   0.   0.   2.]
 [  0.   4.   3.   0.  11.   0.   0.   6.]
 [  0.   3.   3.   0.   7.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   3.]
 [  0.   4.   2.   0.   8.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   0.   0.   0.   0.   0.   0.   8.]
 [  0.   4.   3.   0.  10.   0.   0.   2.]
 [  0.   3.   3.   0.   9.   0.   0.   2.]
 [  0.   4.   3.   0.  11.   0.   0.   6.]
 [  0.   3.   3.   0.   7.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   3.]
 [  0.   4.   2.   0.   8.   0.   0.   2.]
 [  0.   0.   0.   0.   0.   0.   0.   1.]
 [  0.   1.   1.   0.   6.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0.   0.   2.]
 [  0.   0.

In [5]:
# 打印边连接关系（processed_data.edge_index）
print(processed_data.edge_index.numpy())

[[  1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
    1   1   1   2   2   2   2   2   2   2   2   2   2   2   2   2   2   2
    2   2   2   2   2   2   2   2   2   3   3   3   3   3   3   3   3   3
    3   3   3   3   3   3   3   3   3   3   3   3   3   3   4   4   4   4
    4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4   4
    5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5   5
    5   5   5   5   5   5   5   6   6   6   6   6   6   6   6   6   6   6
    6   6   6   6   6   6   6   6   6   6   6   6   6   6   6   7   7   7
    7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7   7
    7   7   7   7   7   7   7   8   8   8   8   8   8   8   8   8   8   8
    8   8   8   8   8   8   8   8   8   8   8   8   8   8   8   9   9   9
    9   9   9   9   9   9   9   9   9   9   9   9   9   9   9   9   9   9
    9   9   9  10  10  10  10  10  10  10  10  10  10  10  10  10  10  10
   10  10  10  10  10  10  10  10  10 

In [6]:
# 打印节点标签（processed_data.y）
print(processed_data.y.numpy())

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0
 0 1 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1 0 1
 0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 